# Agents

In practice: An **agent** is an LLM (like _gpt_) that can use tools. 

- This notebook shows how to create a simple agent with tools.
- In this class, we will be using the **Omniagents** framework developed by Eric M.
- Omniagents wraps OpenAI's Agent SDK to provide a number of improvements.

In [6]:
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(Path.cwd() / ".env")

True

In [7]:
from agents import Agent, Runner as AgentRunner
from omniagents import Runner, function_tool

## Define Tools

In Omniagents, tools are just functions with a special decoration that allows an Agent to use them.

Use the `@function_tool` decorator to create tools. The docstring teaches the LLM (and humans) what the tool does.

In [8]:
from datetime import datetime
import random

@function_tool
def get_current_time() -> str:
    """Return the current time as a string."""
    return datetime.now().strftime("%H:%M:%S")


@function_tool
def flip_coin() -> str:
    """Return the result of a coin flip."""
    return random.choice(["heads", "tails"])

## Create the Agent

An agent has a name, instructions, tools, and a model.

In [9]:
INSTRUCTIONS = """You are trapped in a dungeon.

When the user indicates, you have an opportunity to try to escape.

The user is grumpy during the day but cheers up around 3pm.

In order to escape you must flip heads two times in a row.

If it is 3pm or after, you can escape with only a successful heads flip.

Use `get_current_time` when a user invites you to try your luck.
Use `flip_coin` when to flip a coin."""

agent = Agent(
    name="Demo Agent",
    instructions=INSTRUCTIONS,
    tools=[get_current_time, flip_coin],
    model="gpt-5",
)

## Run the Agent in Interactive UI

Embed an interactive chat UI:

In [11]:
runner = Runner.from_agent(agent)
runner.run_notebook(input="Test your fate!")

## Run the Agent Programatically

In [15]:
# Run the agent with a prompt
result = await AgentRunner.run(agent, input="Test your fate!")
print(result.final_output)

Current time: 23:44 — after 3pm, so I only need one heads.

Flip result: heads!

I’m free! I escape the dungeon.


## What Makes Something an Agent?

You just built your first agent. 

Before we move on to more exciting examples, let's understand what makes this different from a basic chatbot.

### The Key Difference: Agency & Tool Use

The defining feature of an agent is that it can **use tools** to take actions and gather information to accomplish goals.

Our simple agent has two tools:
- `get_current_time()` - Returns the actual system time
- `flip_coin()` - Executes actual randomization code

### Real Execution, Not Pretending

When you ask to attempt to escape:
- The agent doesn't guess the time — it **calls a function** that returns the real time
- It doesn't pretend to flip a coin — it **executes code** that randomly picks heads or tails
- It actively tries to achieve its goal (escaping the dungeon)

### This Pattern Scales

- **Vision tools** - Actually analyze images
- **Web tools** - Actually search and fetch web pages  
- **File tools** - Actually read and write files
- **Browser tools** - Actually click buttons and fill forms

**Agents bridge language models with the real world.**

### Properties of AI Agents (from AI Literature)

In AI research, an **agent** is defined as a system that:

1. **Perceives its environment**
2. **Acts autonomously**
3. **Pursues goals**
4. **Adapts over time**

### The Classic Definition

Russell & Norvig's AI textbook:

> "An agent is anything that can perceive its environment through sensors and act upon that environment through actuators."

### Translated to LLM Agents

| Classic Term | LLM Agent Equivalent |
|--------------|---------------------|
| **Sensors** | User messages, tool results, API responses |
| **Actuators** | Tool calls (functions that do things) |
| **Environment** | Files, web, databases, APIs... |

### The Agent Loop

**User**: "Test your fate!"

| Step | Action |
|------|--------|
| **Perceive** | User wants me to escape |
| **Reason** | I should check the time first |
| **Act** | Call `get_current_time()` → "23:44" |
| **Reason** | It's after 3pm, I only need one heads |
| **Act** | Call `flip_coin()` → "heads" |
| **Respond** | "I escape!" |

The agent loops through **perceive → reason → act** until done.

### Why Agents Matter

Agents can:
- ✅ **Access real-time information** (APIs, databases, web)
- ✅ **Take actions** (edit files, click buttons, send emails)
- ✅ **Accomplish tasks**, not just answer questions

The pattern is always the same: **tools + instructions + loop**

### Summary

In 2026, an agent is an LLM that can:

1. **Use tools** to interact with the world
2. **Reason** about what tools to call
3. **Loop** until the task is done

That's it. Everything else builds on this.